<span style="font-size: 280%;color:#1155cc"> Pytorch_8_Proste sieci neuronowe_autograd

<span style="font-size: 180%;color:red"> 17.09.2020   

https://github.com/jcjohnson/pytorch-examples#pytorch-custom-nn-modules
10:10  10:50
11:05  
Tutaj używamy PyTorch Tensors i autograd do wdrożenia naszej dwuwarstwowej sieci; teraz nie musimy już ręcznie wprowadzać wstecznego przejścia przez sieć:

In [1]:
import torch

Odpalam karte graficzną GPU

In [2]:
#device = torch.device('cpu') # obliczenia robie na CPU
device = torch.device('cuda') # obliczenia robie na GPU

### Dane
- N to wielkość partii (ilośc rekordów, wielkość próbki); 
- D_in jest wymiarem wejściowym (liczba zmiennych opisujących); 
- H jest ukrytym wymiarem (ukryta warstwa); 
- D_out jest wymiarem wyjściowym(ilość zmiennych wynikowych). 


## <span style="color:red">Przyjmuje 10 zmiennych wynikowych! Coś nowego!</span>

In [3]:
N, D_in, H, D_out = 64, 1000, 100, 10

Powyżej zostały określone parametry aby tensor zmiennych niezależnych i tensor wynikowy były odpowiednie 

x: 64 obserwacji i 1000 zmiennych

y: 64 obserwacji i 10 zmiennych

### Tworzy losowe dane wejściowe i wyjściowe 

In [4]:
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

In [5]:
x.shape

torch.Size([64, 1000])

In [6]:
print('x ',x.size())
print('y ',y.size())

x  torch.Size([64, 1000])
y  torch.Size([64, 10])


### Tworzy wagi losowe dla x i y <span style="color:red">(requires_grad=True)</span>

Utwórzenie losowych tensorów dla wag; ustawienie 'requires_grad=True' oznacza, że my
chce obliczyć gradienty dla tych Tensorów podczas przebiegu wstecznego.

In [7]:
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

In [8]:
w1

tensor([[-0.8158,  0.5833,  1.2466,  ...,  2.2299,  1.0695, -0.9622],
        [ 1.5433, -0.2862, -0.0933,  ..., -0.9864,  0.2454, -0.9694],
        [ 2.0560, -0.8016, -0.4107,  ...,  0.0426,  0.0396,  1.0991],
        ...,
        [-0.7856, -1.5273, -0.7680,  ...,  0.2714, -1.3149, -0.6378],
        [ 0.4821,  2.2323, -0.7289,  ..., -0.3109, -0.0324,  1.2442],
        [ 0.2684,  0.4668,  0.1595,  ..., -0.0257, -0.9328, -0.7669]],
       device='cuda:0', requires_grad=True)

In [9]:
w1.shape

torch.Size([1000, 100])

In [12]:
w2.shape

torch.Size([100, 10])

In [10]:
print('w1 dla x: ', w1.shape)
print('w2 dla y: ', w2.shape)

w1 dla x:  torch.Size([1000, 100])
w2 dla y:  torch.Size([100, 10])


### Definiowanie nauki

Przekaz do przodu: oblicza przewidywane y przy użyciu operacji na Tensorach. Ponieważ w1 i w2 wymagają 'requires_grad=True' , operacje obejmujące te Tensory spowodują, że PyTorch zbuduje wykres obliczeniowy, umożliwiając automatyczne obliczanie gradientów. Ponieważ nie wdrażamy już ręcznie przekazywania wstecznego (backward pass), nie musimy przechowywać odniesień do wartości pośrednich.

In [11]:
learning_rate = 1e-6
for t in range(500):

   y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
      # Funkcja błędu r2 dla drukowana dla każdej epoki
   loss = (y_pred - y).pow(2).sum()
   print('Epoka:',t, loss.item())
    
       #KROK 1
       # Użyj programu autograd, aby obliczyć przebieg wsteczny. To połączenie obliczy
       # gradient straty w stosunku do wszystkich Tensorów z 'requires_grad=True'.
       # Po tym wywołaniu w1.grad i w2.grad będą Tensorami trzymającymi gradient
       # straty w odniesieniu odpowiednio do w1 i w2.
   loss.backward()

       #KROK 2
       # Zaktualizuj wagi przy użyciu spadku gradientu. W tym kroku chcemy po prostu mutować
       # wartości w1 i w2 w miejscu; nie chcemy budować obliczeń
       # wykres dla kroków aktualizacji, dlatego używamy menedżera kontekstu torch.no_grad ()
       #, aby zapobiec tworzeniu przez PyTorch wykresu obliczeniowego dla aktualizacji
   with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

       #KROK 3
       # Ręcznie zeruj gradienty po przejściu do tyłu
    w1.grad.zero_()
    w2.grad.zero_()
    
    

Epoka: 0 32573074.0
Epoka: 1 30884630.0
Epoka: 2 35109556.0
Epoka: 3 38711528.0
Epoka: 4 35177040.0
Epoka: 5 24281896.0
Epoka: 6 12736422.0
Epoka: 7 5827867.0
Epoka: 8 2798686.0
Epoka: 9 1606459.125
Epoka: 10 1103126.0
Epoka: 11 850054.75
Epoka: 12 694889.375
Epoka: 13 584889.125
Epoka: 14 499982.625
Epoka: 15 431601.4375
Epoka: 16 375263.09375
Epoka: 17 328201.4375
Epoka: 18 288509.5625
Epoka: 19 254770.65625
Epoka: 20 225913.359375
Epoka: 21 201111.984375
Epoka: 22 179676.90625
Epoka: 23 161021.921875
Epoka: 24 144726.21875
Epoka: 25 130431.8671875
Epoka: 26 117844.015625
Epoka: 27 106752.703125
Epoka: 28 96910.828125
Epoka: 29 88107.5234375
Epoka: 30 80255.3984375
Epoka: 31 73236.40625
Epoka: 32 66944.8828125
Epoka: 33 61290.7265625
Epoka: 34 56194.453125
Epoka: 35 51592.8125
Epoka: 36 47429.59375
Epoka: 37 43654.703125
Epoka: 38 40231.09375
Epoka: 39 37118.28125
Epoka: 40 34282.10546875
Epoka: 41 31694.177734375
Epoka: 42 29326.3203125
Epoka: 43 27159.515625
Epoka: 44 25174.6757812

Epoka: 340 0.009835215285420418
Epoka: 341 0.009437421336770058
Epoka: 342 0.009061599150300026
Epoka: 343 0.008710410445928574
Epoka: 344 0.008360384032130241
Epoka: 345 0.00802975706756115
Epoka: 346 0.007715326268225908
Epoka: 347 0.007410304620862007
Epoka: 348 0.00712016224861145
Epoka: 349 0.006839253939688206
Epoka: 350 0.006573336198925972
Epoka: 351 0.006318126805126667
Epoka: 352 0.006071789655834436
Epoka: 353 0.005835213698446751
Epoka: 354 0.005609195679426193
Epoka: 355 0.0053984010592103004
Epoka: 356 0.005190773867070675
Epoka: 357 0.0049942247569561005
Epoka: 358 0.004805052187293768
Epoka: 359 0.004622099921107292
Epoka: 360 0.004444492980837822
Epoka: 361 0.004281323868781328
Epoka: 362 0.0041229319758713245
Epoka: 363 0.003965272568166256
Epoka: 364 0.0038136644288897514
Epoka: 365 0.003675012616440654
Epoka: 366 0.0035401128698140383
Epoka: 367 0.0034127947874367237
Epoka: 368 0.003287592902779579
Epoka: 369 0.0031700152903795242
Epoka: 370 0.0030558924190700054
Ep